Bayan Nezamabad - 20251971, Euan Bourke - 21332142

In [5]:
# Imports
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
import matplotlib.pyplot as plt
from keras import utils, Sequential

In [6]:
# Hyper Params
EPOCHS = 20
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = optimizers.Adam(0.001)
LOSS = "sparse_categorical_crossentropy"
METRICS = "accuracy"
VALIDATION_SPLIT=0.90

IMG_ROW, IMG_COL = 224, 224 # Image dimensions
INPUT_SHAPE = (IMG_ROW, IMG_COL, 3) # 3 as RGB
NB_CLASSES = 100 # 100 different classifications for 100 different species of butterfly or moth

TRAIN_PATH = 'data/train'
VALID_PATH = 'data/valid'
SEED = 123 # used for consistent randomisation of the dataset

In [7]:
class Dataset:
    def __init__(self):
        self.training_data = utils.image_dataset_from_directory(TRAIN_PATH,
                                                                shuffle=True,
                                                                image_size=(IMG_COL, IMG_ROW),
                                                                batch_size=BATCH_SIZE,
                                                                seed=SEED)
        self.valid_data = utils.image_dataset_from_directory(VALID_PATH,
                                                             shuffle=True,
                                                             image_size=(IMG_COL, IMG_ROW),
                                                             batch_size=BATCH_SIZE,
                                                             seed=SEED)
    
    def load_training_data(self):
        return self.training_data
    
    def load_valid_data(self):
        return self.valid_data
    

data = Dataset()
train_data = data.load_training_data()
valid_data = data.load_valid_data()

Found 12594 files belonging to 100 classes.
Found 500 files belonging to 100 classes.


In [8]:
# Implementation of AlexNet architecture
class AlexNet(Sequential):
    def __init__(self):
        super().__init__()
        self.build_layers()
        self.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[METRICS])

    def build_layers(self):
        # Convolutional layers with batch normalization (and max pooling)
        self.add(layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=INPUT_SHAPE))
        self.add(layers.BatchNormalization())
        self.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

        self.add(layers.Conv2D(256, (5, 5), padding='same', activation='relu'))
        self.add(layers.BatchNormalization())
        self.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

        self.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
        self.add(layers.BatchNormalization())
        self.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
        self.add(layers.BatchNormalization())
        self.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
        self.add(layers.BatchNormalization())

        self.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

        # Flatten layer
        self.add(layers.Flatten())

        # Fully connected layers with dropout
        self.add(layers.Dense(4096, activation='relu'))
        self.add(layers.Dropout(rate=0.5))
        
        self.add(layers.Dense(4096, activation='relu'))
        self.add(layers.Dropout(rate=0.5))
        
        self.add(layers.Dense(1000, activation='relu'))
        
        self.add(layers.Dense(NB_CLASSES, activation='softmax'))

model = AlexNet()
model.summary()

Model: "alex_net_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 54, 54, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 54, 54, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 26, 26, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 26, 26, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 26, 26, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 12, 12, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 384)    │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 12, 12, 384)    │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 5, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │    26,218,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1000)           │     4,097,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 100)            │       100,100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,949,612 (194.36 MB)

 Trainable params: 50,946,860 (194.35 MB)

 Non-trainable params: 2,752 (10.75 KB)

In [9]:
model_summary = model.fit(train_data, epochs=EPOCHS, validation_data=valid_data, verbose=VERBOSE)


Epoch 1/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 98s 971ms/step - accuracy: 0.0290 - loss: 8.2742 - val_accuracy: 0.0140 - val_loss: 4.7793
Epoch 2/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 104s 1s/step - accuracy: 0.1272 - loss: 3.7215 - val_accuracy: 0.0540 - val_loss: 5.0331
Epoch 3/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 119s 1s/step - accuracy: 0.2232 - loss: 3.0360 - val_accuracy: 0.2380 - val_loss: 3.0514
Epoch 4/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.3518 - loss: 2.4656 - val_accuracy: 0.2880 - val_loss: 2.9866
Epoch 5/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 104s 1s/step - accuracy: 0.4398 - loss: 2.1047 - val_accuracy: 0.4500 - val_loss: 2.0683
Epoch 6/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 155s 2s/step - accuracy: 0.4915 - loss: 1.8695 - val_accuracy: 0.5300 - val_loss: 1.7251
Epoch 7/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 158s 2s/step - accuracy: 0.5460 - loss: 1.6599 - val_accuracy: 0.5000 - val_loss: 1.9636
Epoch 8/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 98s 993ms/step - accuracy: 0.5914 - loss: 1.4651 - val_accuracy: 0.5420

Refs: \
AlexNet Architecture: https://www.kaggle.com/code/blurredmachine/alexnet-architecture-a-complete-guide \
https://medium.com/@siddheshb008/alexnet-architecture-explained-b6240c528bd5
Keras.layer docs: https://www.tensorflow.org/api_docs/python/tf/keras/layers/
